In [ ]:
# Lookalike Model
This notebook builds a lookalike model to recommend similar customers based on profile and transaction history.


In [ ]:
import pandas as pd
from sklearn. metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Load data
customers = pd.read_csv('Customers.CSV)
transactions = pd.read_csv('Transactions.CSV)

# Create Customer-Transaction Features
def build_customer_features():
    customer_transactions = transactions.group by('CustomerID').agg({
        'TotalValue': 'sum',
        'Quantity': 'sum',
    }).reset_index()
    merged = pd.merge(customers, customer_transactions, on='CustomerID', how='left')
    merged.fill(0, inplace=True)
    return merged

# Build Lookalike Model
def lookalike_model(customer_data):
    features = customer_data[['TotalValue', 'Quantity']].values
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    similarity_matrix = cosine_similarity(scaled_features)
    lookalikes = {}

    for idx, customer_id in enumerate(customer_data['CustomerID'][:20]):
        scores = [(customer_data['CustomerID'][i], similarity_matrix[idx][i]) for i in range(len(similarity_matrix))]
        scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:4]
        lookalikes[customer_id] = scores

    return lookalikes

# Main Execution
if __name__ == "__main__":
    customer_data = build_customer_features()
    lookalikes = lookalike_model(customer_data)
    lookalike_df = pd.DataFrame([
        {"CustomerID": cust_id, "Lookalikes": str(lookalike)} for cust_id, lookalike in lookalikes.items()
    ])
    lookalike_df.to_csv('./outputs/Lookalike.csv', index=False)
    print("Lookalike Model Completed. Results saved in './outputs/Lookalike.csv'")